In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import gc
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import IPython.display as ipD
import matplotlib.pyplot as plt
import matplotlib.patches as ptc

import tensorflow as tf

from sklearn.preprocessing import StandardScaler, MinMaxScaler

mm = MinMaxScaler()
ss = StandardScaler()

import librosa
import librosa.display as LD

%matplotlib inline

In [ ]:
# paths
test_folder = "../input/rfcx-species-audio-detection/test"
tfrecords = "../input/rfcx-species-audio-detection/tfrecords"
train_folder = "../input/rfcx-species-audio-detection/train"
sample_submission = "../input/rfcx-species-audio-detection/sample_submission.csv"
train_tp = "../input/rfcx-species-audio-detection/train_tp.csv"
train_fp = "../input/rfcx-species-audio-detection/train_fp.csv"

In [ ]:
def seedAll(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"]=str(seed)

seedAll(2021)

In [ ]:
trtp = pd.read_csv(train_tp)
trtp

In [ ]:
plt.figure(figsize=(18, 6))
sns.countplot(x="species_id", data=trtp)
plt.title("Species ID distribution for True Positives")
plt.show()

plt.figure(figsize=(18, 6))
sns.countplot(x="species_id", hue="songtype_id", data=trtp)
plt.title("Species ID distribution for True Positives grouped by song_type")
plt.show()

plt.figure(figsize=(8, 8))
wegdes, texts, autotexts = plt.pie(trtp["songtype_id"].value_counts(), 
        startangle=45, 
        wedgeprops={"linewidth":1, "edgecolor":"black"}, 
        autopct='%1.f%%', 
        shadow=True,
        textprops= dict(color="black"),
        explode=(0.2, 0.2))
plt.legend(wegdes, trtp["songtype_id"].value_counts().index,
          title="song type",
          loc="center",
          bbox_to_anchor=(1, 0, 0, 0))
plt.setp(autotexts, size=14, weight="bold")
plt.title("Song types distribution for True positives")
plt.show()

In [ ]:
trfp = pd.read_csv(train_fp)
trfp

In [ ]:
plt.figure(figsize=(18, 6))
sns.countplot(x="species_id", data=trfp)
plt.title("Species ID distribution for False Positives")
plt.show()

plt.figure(figsize=(18, 6))
sns.countplot(x="species_id", hue="songtype_id", data=trfp)
plt.title("Species ID distribution for False Positives grouped by song_type")
plt.show()

plt.figure(figsize=(8, 8))
wegdes, texts, autotexts = plt.pie(trfp["songtype_id"].value_counts(), 
        startangle=45, 
        wedgeprops={"linewidth":1, "edgecolor":"black"}, 
        autopct='%1.f%%', 
        shadow=True,
        textprops= dict(color="black"),
        explode=(0.2, 0.2))
plt.legend(wegdes, trfp["songtype_id"].value_counts().index,
          title="song type",
          loc="center",
          bbox_to_anchor=(1, 0, 0, 0))
plt.setp(autotexts, size=14, weight="bold")
plt.title("Song types distribution for False positives")
plt.show()

In [ ]:
print("Total Train audio samples: ", len(os.listdir(train_folder)))
print("Total Test audio samples: ", len(os.listdir(test_folder)))
print("Number of samples present both in Train True Positives and Train False Positives: ", len(set(trfp.recording_id.tolist()).intersection(trtp.recording_id.tolist())))
print("Number of unique audio samples in Train True positives: ", trtp.recording_id.nunique())
print("Number of unique audio samples in Train False positives: ", trfp.recording_id.nunique())

In [ ]:
trtp["clip_duration"] = trtp["t_max"] - trtp["t_min"]
trfp["clip_duration"] = trfp["t_max"] - trfp["t_min"]

fig, ax = plt.subplots(1, 2, figsize=(20, 6))
sns.boxplot(x="clip_duration", data=trtp, orient="h", ax=ax[0])
sns.distplot(trtp["clip_duration"], kde=True, ax=ax[1])

fig.suptitle("Distribution of length of audio clips containing any species in True positive")
fig.show()

plt.figure(figsize=(20, 20))
sns.boxplot(x="clip_duration", y="species_id", data=trtp, orient="h")
plt.title("species_id-wise Distribution of length of audio clips containing any species in True positive Samples")
plt.show()

plt.figure(figsize=(20, 20))
sns.boxplot(x="clip_duration", y="species_id", data=trfp, orient="h")
plt.title("species_id-wise Distribution of length of audio clips containing any species in False positive Samples")
plt.show()

In [ ]:
trtp["freq_range"] = trtp["f_max"] - trtp["f_min"]
trfp["freq_range"] = trfp["f_max"] - trfp["f_min"]

fig, ax = plt.subplots(1, 2, figsize=(20, 6))
sns.boxplot(x="freq_range", data=trtp, orient="h", ax=ax[0])
sns.distplot(trtp["freq_range"], kde=True, ax=ax[1])

fig.suptitle("Distribution of Frequency Ranges containing any species in True positive")
fig.show()

plt.figure(figsize=(20, 20))
sns.boxplot(x="freq_range", y="species_id", data=trtp, orient="h")
plt.title("species_id-wise Distribution of Frequency Ranges containing any species in True positive Samples")
plt.show()

plt.figure(figsize=(20, 20))
sns.boxplot(x="freq_range", y="species_id", data=trfp, orient="h")
plt.title("species_id-wise Distribution of Frequency Ranges containing any species in False positive Samples")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(22, 8))

sns.boxplot(y="clip_duration", x="songtype_id", data=trtp, ax=axs[0])
axs[0].set_title("species_id-wise Distribution of length of audio clips containing any species in True positive Samples")

sns.boxplot(y="clip_duration", x="songtype_id", data=trfp, ax=axs[1])
axs[1].set_title("species_id-wise Distribution of length of audio clips containing any species in False positive Samples")

fig.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(22, 8))

sns.boxplot(y="freq_range", x="songtype_id", data=trtp, ax=axs[0])
axs[0].set_title("species_id-wise Distribution of Frequency Ranges containing any species in True positive Samples")

sns.boxplot(y="freq_range", x="songtype_id", data=trfp, ax=axs[1])
axs[1].set_title("species_id-wise Distribution of Frequency Ranges containing any species in False positive Samples")

fig.show()

In [ ]:
# choose a sample from train and test
tr = os.path.join(train_folder, os.listdir(train_folder)[np.random.randint(0, len(os.listdir(train_folder)))])
ts = os.path.join(test_folder, os.listdir(test_folder)[np.random.randint(0, len(os.listdir(test_folder)))])

# load the np array and the samping rate
trx, trsr = librosa.load(tr)
tsx, tssr = librosa.load(ts)
recId_train = (tr.split("/")[-1]).split(".")[0]
recId_test = (ts.split("/")[-1]).split(".")[0]

print("="*10, "Training Sample", "="*10)
display(trfp[trfp["recording_id"]==recId_train] if recId_train in trfp["recording_id"].tolist() else trtp[trtp["recording_id"]==recId_train])
print("="*10, "Test Sample", "="*10)
print("Test Data: ", recId_test)

In [ ]:
# print the shape and the sampling rate
print("=====Train sample=======")
print(trx.shape, trsr)
print("=====Test sample=======")
print(tsx.shape, tssr)

In [ ]:
ipD.Audio(tr)

In [ ]:
ipD.Audio(ts)

In [ ]:
plt.figure(figsize=(14, 5))
LD.waveplot(trx, sr=trsr)
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
LD.waveplot(tsx, sr=tssr)
plt.show()

In [ ]:
TRX = librosa.stft(trx)
print("Shape of the stft: ", TRX.shape)
# convert into db
TRXdb = librosa.amplitude_to_db(abs(TRX))

plt.figure(figsize=(14, 5))
librosa.display.specshow(TRXdb, sr=trsr, x_axis='time', y_axis='hz')
plt.colorbar()
plt.figure(figsize=(14, 5))
librosa.display.specshow(TRXdb, sr=trsr, x_axis='time', y_axis='log')
plt.title("In log scale")
plt.colorbar()
plt.show()

In [ ]:
# convert into fourier transform
TSX = librosa.stft(tsx)
print("Shape of the stft: ", TSX.shape)

# convert into bd
TSXdb = librosa.amplitude_to_db(abs(TSX))
plt.figure(figsize=(14, 5))
librosa.display.specshow(TSXdb, sr=tssr, x_axis='time', y_axis='hz')
plt.colorbar()
plt.figure(figsize=(14, 5))
librosa.display.specshow(TSXdb, sr=tssr, x_axis='time', y_axis='log')
plt.colorbar()
plt.show()

In [ ]:
spectral_centroids = librosa.feature.spectral_centroid(trx, sr=trsr)[0]
print("Shape of the spectral centroids: ", spectral_centroids.shape)

# extract the time and frame indices
frames = range(len(spectral_centroids))
t = librosa.frames_to_time(frames)

plt.figure(figsize=(10, 4))
LD.waveplot(trx, sr=trsr, alpha=0.4)
plt.title("Spectral Centroids for Train sample")
plt.show()

plt.figure(figsize=(10, 4))
LD.waveplot(trx, sr=trsr, alpha=0.4)
plt.plot(t, ss.fit_transform(spectral_centroids.reshape(-1, 1)))
plt.plot(t, mm.fit_transform(spectral_centroids.reshape(-1, 1)))
plt.legend(["Audio Signal", "sc_ss", "sc_mm"][::-1])
plt.title("Normalized Spectral Centroid Visualization for Train sample")
plt.show()

In [ ]:
spectral_centroids = librosa.feature.spectral_centroid(tsx, sr=tssr)[0]
print("Shape of the spectral centroids: ", spectral_centroids.shape)

# extract the time and frame indices
frames = range(len(spectral_centroids))
t = librosa.frames_to_time(frames)

plt.figure(figsize=(10, 4))
LD.waveplot(tsx, sr=tssr, alpha=0.4)
plt.title("Spectral Centroids for Test sample")
plt.show()

plt.figure(figsize=(10, 4))
LD.waveplot(tsx, sr=tssr, alpha=0.4)
plt.plot(t, ss.fit_transform(spectral_centroids.reshape(-1, 1)))
plt.plot(t, mm.fit_transform(spectral_centroids.reshape(-1, 1)))
plt.legend(["Audio Signal", "sc_ss", "sc_mm"][::-1])
plt.title("Normalized Spectral Centroid Visualization for Test sample")
plt.show()

In [ ]:
spectral_rolloff = librosa.feature.spectral_rolloff(trx+0.01, sr=trsr)
frames = range(len(spectral_rolloff))
t = librosa.frames_to_time(frames)

plt.figure(figsize=(12, 4))
LD.waveplot(trx, sr=trsr, alpha=0.4)
plt.plot(t, ss.fit_transform(spectral_rolloff), color='r')
plt.show()

In [ ]:
spectral_rolloff = librosa.feature.spectral_rolloff(tsx, sr=tssr)
frames = range(len(spectral_rolloff))
t = librosa.frames_to_time(frames)

plt.figure(figsize=(10, 4))
LD.waveplot(tsx, sr=tssr, alpha=0.4)
plt.plot(t, ss.fit_transform(spectral_rolloff), color="r")
plt.show()

In [ ]:
mfccs = librosa.feature.mfcc(trx, sr=trsr)
print(mfccs.shape)
#Displaying  the MFCCs:
plt.figure(figsize=(15, 7))
LD.specshow(mfccs, sr=trsr, x_axis='time')
plt.show()

In [ ]:
mfccs = librosa.feature.mfcc(tsx, sr=tssr)
print(mfccs.shape)
#Displaying  the MFCCs:
plt.figure(figsize=(15, 7))
LD.specshow(mfccs, sr=tssr, x_axis='time')
plt.show()

In [ ]:
melspec = librosa.feature.melspectrogram(trx, sr=trsr)
print(melspec.shape)
plt.figure(figsize=(10, 4))
librosa.display.specshow(librosa.power_to_db(melspec, ref=np.max),
                         y_axis='mel',
                         x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel spectrogram')
plt.tight_layout()

In [ ]:
hop_length=12
chromagram = librosa.feature.chroma_stft(trx, sr=trsr, hop_length=hop_length)
plt.figure(figsize=(15, 5))
LD.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=hop_length, cmap='coolwarm')
plt.show()

In [ ]:
chromagram = librosa.feature.chroma_stft(tsx, sr=tssr, hop_length=hop_length)
plt.figure(figsize=(15, 5))
LD.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=hop_length, cmap='coolwarm')
plt.show()

In [ ]:
samSub = pd.read_csv(sample_submission)
samSub

In [ ]:
train_tfrec = "../input/rfcx-species-audio-detection/tfrecords/train"
test_tfrec = "../input/rfcx-species-audio-detection/tfrecords/test"

train_tfrecs = sorted(tf.io.gfile.glob(train_tfrec + "/*.tfrec"))
test_tfrecs = sorted(tf.io.gfile.glob(test_tfrec + "/*.tfrec"))

print("Number of train tfrecords: ", len(train_tfrecs))
print("Number of test tfrecords: ", len(test_tfrecs))

In [ ]:
sample_proto_train_tfrec = tf.data.TFRecordDataset([train_tfrecs[0]])
sample_proto_train_tfrec

In [ ]:
print("Number of samples in one single record: ", sample_proto_train_tfrec.reduce(np.int64(), lambda x, _: x+1).numpy())

In [ ]:
CUT_TIME = 10 # cutting window in seconds
SAMPLE_TIME = 6
SR = 48000
FMAX = 24000
FMIN = 40
# feature description for the tfrecords
# this will parsed as arguments into tf.io.parse_single_example
feature_description = {
    'recording_id': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'audio_wav': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'label_info': tf.io.FixedLenFeature([], tf.string, default_value=''),
}
parse_dtype = {
    'audio_wav': tf.float32,
    'recording_id': tf.string,
    'species_id': tf.int32,
    'songtype_id': tf.int32,
    't_min': tf.float32,
    'f_min': tf.float32,
    't_max': tf.float32,
    'f_max':tf.float32,
    'is_tp': tf.int32
}

# define a fun to read the encoded tfrec
@tf.function
def _parse_fun(sample):
    sample = tf.io.parse_single_example(sample, feature_description) # this returns a dicionary of the features for a single tfrec
    audio, _ = tf.audio.decode_wav(sample["audio_wav"], desired_channels=1)
    label_info = tf.strings.split(sample["label_info"], ";")
    labels = tf.strings.regex_replace(label_info, '"', '')
    
    @tf.function
    def _cut_audio(label):
        items = tf.strings.split(label, sep=',')
        spid = tf.squeeze(tf.strings.to_number(items[0], tf.int32))
        soid = tf.squeeze(tf.strings.to_number(items[1], tf.int32))
        tmin = tf.squeeze(tf.strings.to_number(items[2]))
        fmin = tf.squeeze(tf.strings.to_number(items[3]))
        tmax = tf.squeeze(tf.strings.to_number(items[4]))
        fmax = tf.squeeze(tf.strings.to_number(items[5]))
        tp = tf.squeeze(tf.strings.to_number(items[6], tf.int32))

        tmax_s = tmax * tf.cast(SR, tf.float32)
        tmin_s = tmin * tf.cast(SR, tf.float32)
        cut_s = tf.cast(CUT_TIME * SR, tf.float32)
        all_s = tf.cast(60 * SR, tf.float32)
        tsize_s = tmax_s - tmin_s
        cut_min = tf.cast(
            tf.maximum(0.0, 
                tf.minimum(tmin_s - (cut_s - tsize_s) / 2,
                           tf.minimum(tmax_s + (cut_s - tsize_s) / 2,
                                      all_s) - cut_s)
            ), tf.int32
        )
        cut_max = cut_min + CUT_TIME * SR
        
        _sample = {
            'audio_wav': tf.reshape(audio[cut_min:cut_max], [CUT_TIME*SR]),
            'recording_id': sample['recording_id'],
            'species_id': spid,
            'songtype_id': soid,
            't_min': tmin - tf.cast(cut_min, tf.float32)/tf.cast(SR, tf.float32),
            'f_min': fmin,
            't_max': tmax - tf.cast(cut_min, tf.float32)/tf.cast(SR, tf.float32),
            'f_max': fmax,
            'is_tp': tp
        }
        return _sample
    
    samples = tf.map_fn(_cut_audio, labels, dtype=parse_dtype)
    return samples
    pass

In [ ]:
parsed_tfrecs_sample = sample_proto_train_tfrec.map(_parse_fun).unbatch()


In [ ]:
sample = next(iter(parsed_tfrecs_sample))

for key, val in sample.items():
    print(key, ":", val)

In [ ]:
@tf.function
def cut_audio(sample, istrain=True):
    # random cutting for train samples
    if istrain:
        cut_min = tf.random.uniform([],
                                    maxval=(CUT_TIME-SAMPLE_TIME) * SR,
                                    dtype=tf.int32)
    else:
        # center cropping for validation data
        cut_min = (CUT_TIME - SAMPLE_TIME) * SR//2
    cut_max = cut_min + SAMPLE_TIME * SR
    cutaudio = tf.reshape(
        sample["audio_wav"][cut_min:cut_max], [SAMPLE_TIME * SR]
    )

    result = {}
    result.update(sample)
    result["audio_wav"] = cutaudio
    result["t_min"] = tf.maximum(0.0, sample["t_min"] - tf.cast(cut_min, tf.float32)/SR)
    result["t_max"] = tf.maximum(0.0, sample["t_max"] - tf.cast(cut_min, tf.float32)/SR)
    return result
    pass

In [ ]:
@tf.function
def waveToSpec(sample):
    mel_power = 2
    stfts = tf.signal.stft(sample["audio_wav"],
                           frame_length=2048,
                           frame_step=512,
                           fft_length=2048)
    spectograms = tf.abs(stfts) ** mel_power

    # convert into mel scale
    mel_weight = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=224,  # or can be said as the no of MFCCs, though theoretically, ideal value should be in 30-50 range, let's try with 224 for image size
        num_spectrogram_bins=stfts.shape[-1],
        sample_rate=SR,
        lower_edge_hertz=FMIN,
        upper_edge_hertz=FMAX
    )
    mel_spectrograms = tf.tensordot(
        spectograms, mel_weight, 1
    )
    mel_spectrograms.set_shape(spectograms.shape[:-1].concatenate(mel_weight.shape[-1:]))
    log_mel_spectograms = tf.math.log(mel_spectrograms + 1e-6)

    results = {
        "audio_spec": tf.transpose(log_mel_spectograms)  # of shape (num_mel_spec_bins, num_frames)
    }
    results.update(sample)
    return results
    pass

In [ ]:
@tf.function
def filter_tp(sample):
    """

    :param sample: Processed dictionary from _parse_function
    :return: boolean, whether belongs to true positive or false positive
    """
    return sample["is_tp"] == 1
    pass


In [ ]:
@tf.function
def create_annot(sample):
    target = tf.one_hot(sample["species_id"],
                        24,
                        on_value=sample["is_tp"],
                        off_value=0)

    return {
        "input": sample["audio_spec"],  # obtained from creating spectograms from audio np arrays
        "target": tf.cast(target, tf.float32)
    }
    pass

In [ ]:
@tf.function
def toImage(logmelSpec):
    # expand one dimension axis to be treated as image
    image = tf.expand_dims(logmelSpec, axis=-1)
    image = tf.image.resize(image, (224, 512))
    image = tf.image.per_image_standardization(image)

    # no augmentation at this stage
    image = (image - tf.reduce_mean(image))/(tf.reduce_max(image) * tf.reduce_min(image)) * 255.0
    image = tf.image.grayscale_to_rgb(image)
    # image = cfg.model_params["preprocess"](image)
    return image
    pass


@tf.function
def preprocess_img(sample):
    image = toImage(sample["input"])
    return image, sample["target"]
    pass

In [ ]:
# let's extract the samples from the tfrecs and create our initial processed dataset
spec_dataset = parsed_tfrecs_sample.filter(filter_tp).map(cut_audio).map(waveToSpec)
img_dataset = spec_dataset.map(create_annot).map(preprocess_img)

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(20, 10))
for i, s in enumerate(spec_dataset.take(3)):
    axs[0, i].imshow(s['audio_spec'])
    axs[0, i].set_title(s['recording_id'].numpy().decode("UTF-8") + "->" + "s-" +str(s["species_id"].numpy()))
    LD.waveplot(s["audio_wav"].numpy(), sr=SR, ax=axs[1, i])
    LD.specshow(s['audio_spec'].numpy(), x_axis="time", y_axis="mel", sr=SR, fmax=FMAX, fmin=FMIN, ax=axs[2, i], cmap="magma")
    axs[2, i].add_patch(ptc.Rectangle(xy=(s["t_min"], s["f_min"]), height=s["f_max"]-s["f_min"], width=s["t_max"]-s["t_min"], fill=False))
    axs[2, i].text(s["t_min"], s["f_min"], "s-" +str(s["species_id"].numpy()), horizontalalignment='left', verticalalignment='bottom', fontsize=16)                   
plt.show()